# Лабораторная работа №6

*Кищенко Екатерина  
ПИН-212*

## Задание 
>Провести классификацию найденного датасета, методами CatBoost

**Подготовка к обработке данных**

Подключение и преднастройка необходимых компонент

Импорт библиотек

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_squared_error
from catboost import CatBoostClassifier

**Загрузка датасета**

Загрузка датасета

In [8]:
df = pd.read_csv('metal.csv', encoding='utf-8')
df['Mode']= df['Mode'].map({1: True, 0: False}).astype(bool)
print(df.head(3))
print(df.dtypes)

               Spotify ID              Artist IDs Track Name  \
0  19vHgVS1aukRiQWhTqfKnE  1Ffb6ejR6Fe5IamqA5oRUF   DArkSide   
1  5BgnL6gHauuvxe4Ok6W1aC  1Ffb6ejR6Fe5IamqA5oRUF       LosT   
2  0M3adYbGtyRHACP86dey1H  1Ffb6ejR6Fe5IamqA5oRUF     Throne   

          Album Name        Artist Name(s) Release Date  Duration (ms)  \
0           DArkSide  Bring Me The Horizon   10/18/2023         164790   
1               LosT  Bring Me The Horizon     5/4/2023         205368   
2  That's The Spirit  Bring Me The Horizon    9/11/2015         191520   

   Popularity                                Added By              Added At  \
0          74  spotify:user:woth5clwe5p856iw52alp7uek  2023-10-25T13:10:28Z   
1          72  spotify:user:woth5clwe5p856iw52alp7uek  2023-10-25T13:10:28Z   
2          76  spotify:user:woth5clwe5p856iw52alp7uek  2023-10-25T13:10:28Z   

   ... Key  Loudness   Mode  Speechiness  Acousticness  Instrumentalness  \
0  ...   7    -3.018   True       0.1460      0.00035

Удаление ненужных столбцов из датасета

In [9]:
df = df.drop(['Spotify ID', 'Added By', 'Added At', 'Time Signature'], axis=1)
print(df.head(3))

               Artist IDs Track Name         Album Name        Artist Name(s)  \
0  1Ffb6ejR6Fe5IamqA5oRUF   DArkSide           DArkSide  Bring Me The Horizon   
1  1Ffb6ejR6Fe5IamqA5oRUF       LosT               LosT  Bring Me The Horizon   
2  1Ffb6ejR6Fe5IamqA5oRUF     Throne  That's The Spirit  Bring Me The Horizon   

  Release Date  Duration (ms)  Popularity  \
0   10/18/2023         164790          74   
1     5/4/2023         205368          72   
2    9/11/2015         191520          76   

                                              Genres  Danceability  Energy  \
0  melodic metalcore,metalcore,modern rock,rock,u...         0.387   0.940   
1  melodic metalcore,metalcore,modern rock,rock,u...         0.396   0.939   
2  melodic metalcore,metalcore,modern rock,rock,u...         0.426   0.888   

   Key  Loudness   Mode  Speechiness  Acousticness  Instrumentalness  \
0    7    -3.018   True       0.1460      0.000352          0.000003   
1    9    -3.155  False       0.2270 

Удаление строк с пропущенными значениями

In [10]:
df.dropna(inplace=True)

In [11]:
# Кодирование категориальных признаков
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])
    
# Масштабирование числовых признаков
scaler = StandardScaler()
numeric_features = df.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
df[numeric_features] = scaler.fit_transform(df[numeric_features])

Разделение данных на обучающий и тестовый наборы

In [12]:
X = df.drop('Mode', axis=1)
Y = df['Mode']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#### По-умолчанию

Создание и обучение модели

In [13]:
%%time

# создаем модель
catboost = CatBoostClassifier()

# подаем на вход модели обучающие данные
catboost.fit(X_train, Y_train)

Learning rate set to 0.027963
0:	learn: 0.6887058	total: 73.8ms	remaining: 1m 13s
1:	learn: 0.6845754	total: 90.6ms	remaining: 45.2s
2:	learn: 0.6814591	total: 108ms	remaining: 36s
3:	learn: 0.6782007	total: 125ms	remaining: 31.2s
4:	learn: 0.6753265	total: 143ms	remaining: 28.5s
5:	learn: 0.6718025	total: 160ms	remaining: 26.6s
6:	learn: 0.6686017	total: 177ms	remaining: 25.2s
7:	learn: 0.6652565	total: 197ms	remaining: 24.5s
8:	learn: 0.6622100	total: 218ms	remaining: 24s
9:	learn: 0.6596090	total: 233ms	remaining: 23.1s
10:	learn: 0.6569498	total: 248ms	remaining: 22.3s
11:	learn: 0.6542439	total: 262ms	remaining: 21.6s
12:	learn: 0.6515567	total: 276ms	remaining: 21s
13:	learn: 0.6491311	total: 291ms	remaining: 20.5s
14:	learn: 0.6468566	total: 304ms	remaining: 20s
15:	learn: 0.6453031	total: 318ms	remaining: 19.6s
16:	learn: 0.6431147	total: 331ms	remaining: 19.1s
17:	learn: 0.6411886	total: 349ms	remaining: 19.1s
18:	learn: 0.6393228	total: 364ms	remaining: 18.8s
19:	learn: 0.637

164:	learn: 0.5745061	total: 2.64s	remaining: 13.3s
165:	learn: 0.5743437	total: 2.65s	remaining: 13.3s
166:	learn: 0.5740505	total: 2.68s	remaining: 13.4s
167:	learn: 0.5738203	total: 2.7s	remaining: 13.4s
168:	learn: 0.5735671	total: 2.71s	remaining: 13.3s
169:	learn: 0.5732687	total: 2.72s	remaining: 13.3s
170:	learn: 0.5730242	total: 2.73s	remaining: 13.3s
171:	learn: 0.5727934	total: 2.75s	remaining: 13.2s
172:	learn: 0.5725666	total: 2.77s	remaining: 13.2s
173:	learn: 0.5722968	total: 2.79s	remaining: 13.2s
174:	learn: 0.5720309	total: 2.8s	remaining: 13.2s
175:	learn: 0.5718171	total: 2.82s	remaining: 13.2s
176:	learn: 0.5715447	total: 2.84s	remaining: 13.2s
177:	learn: 0.5713804	total: 2.86s	remaining: 13.2s
178:	learn: 0.5711148	total: 2.88s	remaining: 13.2s
179:	learn: 0.5708649	total: 2.9s	remaining: 13.2s
180:	learn: 0.5705747	total: 2.92s	remaining: 13.2s
181:	learn: 0.5704023	total: 2.94s	remaining: 13.2s
182:	learn: 0.5702026	total: 2.95s	remaining: 13.2s
183:	learn: 0.5

331:	learn: 0.5373671	total: 4.86s	remaining: 9.79s
332:	learn: 0.5370829	total: 4.88s	remaining: 9.78s
333:	learn: 0.5368908	total: 4.9s	remaining: 9.77s
334:	learn: 0.5367007	total: 4.92s	remaining: 9.77s
335:	learn: 0.5364993	total: 4.94s	remaining: 9.77s
336:	learn: 0.5362632	total: 4.96s	remaining: 9.76s
337:	learn: 0.5360744	total: 4.98s	remaining: 9.75s
338:	learn: 0.5357395	total: 4.99s	remaining: 9.74s
339:	learn: 0.5356171	total: 5s	remaining: 9.71s
340:	learn: 0.5353790	total: 5.01s	remaining: 9.69s
341:	learn: 0.5351868	total: 5.02s	remaining: 9.67s
342:	learn: 0.5349004	total: 5.04s	remaining: 9.64s
343:	learn: 0.5346233	total: 5.04s	remaining: 9.62s
344:	learn: 0.5344171	total: 5.05s	remaining: 9.6s
345:	learn: 0.5342034	total: 5.07s	remaining: 9.58s
346:	learn: 0.5339519	total: 5.08s	remaining: 9.56s
347:	learn: 0.5336923	total: 5.09s	remaining: 9.54s
348:	learn: 0.5334605	total: 5.1s	remaining: 9.52s
349:	learn: 0.5332160	total: 5.12s	remaining: 9.5s
350:	learn: 0.53304

504:	learn: 0.4976631	total: 7.3s	remaining: 7.15s
505:	learn: 0.4975283	total: 7.31s	remaining: 7.13s
506:	learn: 0.4973537	total: 7.32s	remaining: 7.12s
507:	learn: 0.4971347	total: 7.33s	remaining: 7.1s
508:	learn: 0.4968672	total: 7.34s	remaining: 7.08s
509:	learn: 0.4966221	total: 7.36s	remaining: 7.07s
510:	learn: 0.4964117	total: 7.37s	remaining: 7.05s
511:	learn: 0.4962734	total: 7.38s	remaining: 7.03s
512:	learn: 0.4961240	total: 7.39s	remaining: 7.01s
513:	learn: 0.4959195	total: 7.4s	remaining: 7s
514:	learn: 0.4957706	total: 7.41s	remaining: 6.98s
515:	learn: 0.4955387	total: 7.42s	remaining: 6.96s
516:	learn: 0.4952825	total: 7.43s	remaining: 6.94s
517:	learn: 0.4951144	total: 7.44s	remaining: 6.92s
518:	learn: 0.4949327	total: 7.45s	remaining: 6.9s
519:	learn: 0.4947162	total: 7.46s	remaining: 6.89s
520:	learn: 0.4945557	total: 7.47s	remaining: 6.87s
521:	learn: 0.4942785	total: 7.48s	remaining: 6.85s
522:	learn: 0.4940699	total: 7.49s	remaining: 6.83s
523:	learn: 0.49383

673:	learn: 0.4636761	total: 9.54s	remaining: 4.62s
674:	learn: 0.4634597	total: 9.59s	remaining: 4.62s
675:	learn: 0.4632392	total: 9.62s	remaining: 4.61s
676:	learn: 0.4630640	total: 9.64s	remaining: 4.6s
677:	learn: 0.4628710	total: 9.66s	remaining: 4.59s
678:	learn: 0.4627091	total: 9.67s	remaining: 4.57s
679:	learn: 0.4625200	total: 9.69s	remaining: 4.56s
680:	learn: 0.4623442	total: 9.7s	remaining: 4.54s
681:	learn: 0.4621584	total: 9.71s	remaining: 4.53s
682:	learn: 0.4620239	total: 9.72s	remaining: 4.51s
683:	learn: 0.4618343	total: 9.73s	remaining: 4.49s
684:	learn: 0.4615866	total: 9.74s	remaining: 4.48s
685:	learn: 0.4613710	total: 9.75s	remaining: 4.46s
686:	learn: 0.4611891	total: 9.76s	remaining: 4.45s
687:	learn: 0.4610432	total: 9.77s	remaining: 4.43s
688:	learn: 0.4608106	total: 9.78s	remaining: 4.42s
689:	learn: 0.4606321	total: 9.79s	remaining: 4.4s
690:	learn: 0.4604549	total: 9.8s	remaining: 4.38s
691:	learn: 0.4602561	total: 9.81s	remaining: 4.37s
692:	learn: 0.46

844:	learn: 0.4334138	total: 11.5s	remaining: 2.12s
845:	learn: 0.4331613	total: 11.6s	remaining: 2.1s
846:	learn: 0.4330407	total: 11.6s	remaining: 2.09s
847:	learn: 0.4328745	total: 11.6s	remaining: 2.08s
848:	learn: 0.4326881	total: 11.6s	remaining: 2.06s
849:	learn: 0.4324172	total: 11.6s	remaining: 2.05s
850:	learn: 0.4322596	total: 11.6s	remaining: 2.03s
851:	learn: 0.4320915	total: 11.6s	remaining: 2.02s
852:	learn: 0.4318872	total: 11.6s	remaining: 2.01s
853:	learn: 0.4317257	total: 11.7s	remaining: 1.99s
854:	learn: 0.4315867	total: 11.7s	remaining: 1.98s
855:	learn: 0.4314151	total: 11.7s	remaining: 1.97s
856:	learn: 0.4312561	total: 11.7s	remaining: 1.95s
857:	learn: 0.4310499	total: 11.7s	remaining: 1.94s
858:	learn: 0.4308889	total: 11.7s	remaining: 1.93s
859:	learn: 0.4307271	total: 11.8s	remaining: 1.91s
860:	learn: 0.4305274	total: 11.8s	remaining: 1.9s
861:	learn: 0.4303100	total: 11.8s	remaining: 1.89s
862:	learn: 0.4301776	total: 11.8s	remaining: 1.88s
863:	learn: 0.

Вывод результата

In [14]:
# предсказываем результат на тестовой выборке
catboost_pred = catboost.predict(X_test)

# оцениваем модель
print(f'Сравнение метрик: \n{classification_report(Y_test, catboost_pred)}\n')
print(f'Матрица ошибок: \n{confusion_matrix(Y_test, catboost_pred)}\n')

Сравнение метрик: 
              precision    recall  f1-score   support

       False       0.63      0.47      0.54       991
        True       0.72      0.83      0.77      1601

    accuracy                           0.69      2592
   macro avg       0.67      0.65      0.65      2592
weighted avg       0.68      0.69      0.68      2592


Матрица ошибок: 
[[ 463  528]
 [ 273 1328]]



#### С кросс-валидацией

Проведение процедуры кросс-валидации

In [15]:
%%time

# задаем диапазон параметров
# задаем параметры
parameters = {
    'depth': [5, 10], # максимальная глубина дерева
    'learning_rate': [0.01, 0.1], # скорость обучения
    'iterations': [10, 100], # количество итераций
    'l2_leaf_reg': [1, 10], # коэффициент L2 регуляризации листьев
    'border_count': [1, 255], # количество границ для числовых признаков
    'loss_function': ['Logloss', 'CrossEntropy', 'MultiClass', 'MultiClassOneVsAll'], # функция потерь
    'random_strength': [0, 1], # сила случайности в выборе признаков на каждом уровне
    'bagging_temperature': [0, 1], # температура для баггинга
    'od_type': ['IncToDec', 'Iter', 'None'], # тип оптимизации
    'od_wait': [10, 100] # количество итераций между проверками оптимизации
}

# подбираем лучшие параметры
grid_catboost = RandomizedSearchCV(
    CatBoostClassifier(),
    parameters,
    scoring='f1',
    n_jobs = -1
)
# обучаем модель
grid_catboost.fit(X_train, Y_train)

0:	learn: 0.6863800	total: 3.5ms	remaining: 31.5ms
1:	learn: 0.6809440	total: 7.15ms	remaining: 28.6ms
2:	learn: 0.6763761	total: 10.7ms	remaining: 25ms
3:	learn: 0.6723699	total: 14.5ms	remaining: 21.8ms
4:	learn: 0.6690561	total: 18.6ms	remaining: 18.6ms
5:	learn: 0.6662137	total: 22.1ms	remaining: 14.8ms
6:	learn: 0.6636305	total: 25.7ms	remaining: 11ms
7:	learn: 0.6615362	total: 29.4ms	remaining: 7.34ms
8:	learn: 0.6597398	total: 33.2ms	remaining: 3.69ms
9:	learn: 0.6579931	total: 37ms	remaining: 0us
CPU times: total: 672 ms
Wall time: 1min 19s


RandomizedSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x000002BCA8A56190>,
                   n_jobs=-1,
                   param_distributions={'bagging_temperature': [0, 1],
                                        'border_count': [1, 255],
                                        'depth': [5, 10],
                                        'iterations': [10, 100],
                                        'l2_leaf_reg': [1, 10],
                                        'learning_rate': [0.01, 0.1],
                                        'loss_function': ['Logloss',
                                                          'CrossEntropy',
                                                          'MultiClass',
                                                          'MultiClassOneVsAll'],
                                        'od_type': ['IncToDec', 'Iter', 'None'],
                                        'od_wait': [10, 100],
                                        'random_strength': [0, 1]},
                   scoring='f1')

Вывод результата кросс-валидации

In [17]:
# результаты кросс-валидации
print(f'Лучшие параметры:\n{grid_catboost.best_params_}\n')
print("Показатель f1 для лучшей модели составил: {:.2f}%".
      format(grid_catboost.best_score_ * 100) + '\n')

# предсказываем результат на тестовой выборке
grid_catboost_pred = grid_catboost.predict(X_test)

# оцениваем модель
print(f'Сравнение метрик: \n{classification_report(Y_test, grid_catboost_pred)}\n')
print(f'Матрица ошибок: \n{confusion_matrix(Y_test, grid_catboost_pred)}\n')

Лучшие параметры:
{'random_strength': 0, 'od_wait': 100, 'od_type': 'Iter', 'loss_function': 'CrossEntropy', 'learning_rate': 0.1, 'l2_leaf_reg': 1, 'iterations': 10, 'depth': 5, 'border_count': 1, 'bagging_temperature': 0}

Показатель f1 для лучшей модели составил: 76.51%

Сравнение метрик: 
              precision    recall  f1-score   support

       False       0.53      0.03      0.06       991
        True       0.62      0.98      0.76      1601

    accuracy                           0.62      2592
   macro avg       0.57      0.51      0.41      2592
weighted avg       0.58      0.62      0.49      2592


Матрица ошибок: 
[[  30  961]
 [  27 1574]]

